In [34]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
from db.database import DatabaseConnector
from dtloader.dataloader import DataLoader
from grangercausality_test import GrangerCausalityTest
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.tsatools import lagmat2ds
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from scipy import stats
from utils.utils import *
from statsmodels.tsa.vector_ar.var_model import VAR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dl = DataLoader()
dl.load('sample.log')
dl.extractinfo(export=False,single_file=True)

#query
currtot = pd.DataFrame(dl.full_dict[('sample.log','CURR')])['CurrTot']
desroll = pd.DataFrame(dl.full_dict[('sample.log','ATT')])['DesRoll']

mongodb://localhost:27017/
Database connected


In [35]:
ts = {'ATT':['DesRoll'],'RCIN':['C1']}
dft = corr_var('sample.log',dl,ts,find_corr=False)
dft.dropna(axis=0,how='any',inplace=True)

In [5]:
_, desroll_stat = gc.stationary_test(dft['ATT_DesRoll'],plot=False,print_stats=False)
_, rcin_stat = gc.stationary_test(dft['RCIN_C1'],plot=False,print_stats=False)
if desroll_stat and rcin_stat:
    print('Both Stationary')

NameError: name 'gc' is not defined

In [ ]:
a  = np.array(dft)

In [ ]:
your_permutation = [0,1]
i = np.argsort(your_permutation)
s = grangercausalitytests(a[:,i],5)

In [45]:
gc = GrangerCausalityTest(pd_frame=dft,maxlag=10)

In [46]:
gc.var_grangertest()

In [47]:
gc.dict_res

{'ATT_DesRoll causes RCIN_C1': True,
 'RCIN_C1 causes ATT_DesRoll': True,
 'no-causal': False,
 'bc': True,
 'err': nan}

In [ ]:
dl = DataLoader()
dl.load('sample.log')
dl.extractinfo(export=False,single_file=True)

In [ ]:
currtot = pd.DataFrame(dl.full_dict[('sample.log','CURR')])['CurrTot']
desroll = pd.DataFrame(dl.full_dict[('sample.log','ATT')])['DesRoll']

In [ ]:
gc = GrangerCausalityTest()
df,res = gc.stationary_test(currtot,plot=False,print_stats=False)
if res:
    print('The series is stationary')
else:
    print('The series is not stationary')

In [ ]:
def test_causality(self,var_affected,var_cause,maxlag):
    pass

In [ ]:
grangercausalitytests

In [ ]:
np.nan ==np.nan

In [ ]:
if np.isnan(np.nan):
    print('sds')